# Polygon Overlape and road overlape check

In [13]:
import geopandas as gpd
import pandas as pd

def mark_overlapping_polygons(gdf_1,gdf_2):
    if (gdf_1.crs =='EPSG:4326' and gdf_2.crs == 'EPSG:4326') is False:
        gdf_1 = gdf_1.to_crs('EPSG:4326')
        gdf_2 = gdf_2.to_crs('EPSG:4326')

    gdf_1.reset_index(drop=True,inplace=True)
    gdf_2.reset_index(drop=True,inplace=True)
    # Add a new column to mark overlapping polygons
    gdf_1['poly_overlap'] = 0

    # Build the spatial index
    spatial_index = gdf_1.sindex

    # Check for overlapping polygons using the spatial index
    for i, geometry1 in enumerate(gdf_1.geometry):
        if gdf_1.at[i, 'poly_overlap'] == 1:
            continue  # Skip if the polygon is already marked as overlapping
        possible_overlaps = list(spatial_index.intersection(geometry1.bounds))
        for j in possible_overlaps:
            if i != j and geometry1.intersects(gdf_1.at[j, 'geometry']):
                # Mark only the first overlapping polygon in the 'overlapping' column
                gdf_1.at[i, 'poly_overlap'] = 1
                gdf_1.at[j, 'poly_overlap'] = 1  # Mark the other as 0
                break  # Stop checking other possible overlaps
    intersection = gpd.overlay(gdf_1, gdf_2, how='intersection')
    gdf_1['road_overlap'] = 0
    gdf_1.loc[gdf_1.intersects(gdf_2.unary_union),'road_overlap'] = 1

    return gdf_1

In [ ]:
# Give both the shapefile path 
GT_path = "E:/RnD/Task/GT_Cleaning/KAMAREDDY/GT_Polygons/kamareddy.shp"
road_osm = "E:/RnD/Task/GT_Cleaning/KAMAREDDY/GT_Polygons/Road_kamreddy.shp"
# This will save the QC_checked file into GT folder path.. 
mark_overlapping_polygons(gpd.read_file(GT_path),gpd.read_file(road_osm)).to_file(GT_path.split('.')[0]+'_QC_Checked.shp')

In [ ]:
# This will remove overlapping polygons and save it to GT folder path
cleaned_file = mark_overlapping_polygons(gpd.read_file(GT_path),gpd.read_file(road_osm))
cleaned_file.loc[(cleaned_file['poly_overlap']==0) & (cleaned_file['road_overlap']==0)].to_file(GT_path.split('.')[0]+'_QC_Cleaned.shp')